# GLBL 5050: Introduction to Python for Global Affairs

## Final Project: An Analytics Dashboard for Examining the Economic Dynamics of Countries that have Undergone Democracy Backsliding
#### December 17, 2023
#### Rakkshet Singhaal

###### Summary

Based on the concepts and skills learnt over the last semester, our team has developed a democracy backsliding economic analytics dashboard! The tool is designed to provide a comprehensive economic analysis of countries that have witnessed a regression in their democratic principles. Through intuitive visualizations and data-driven insights, our dashboard aims to shed light on the economic landscape amidst democracy backsliding. 

###### Methodology 

Our team embarked on a systematic process to reach our final outcome. Beginning with the utilization of the V-Dem dataset, we meticulously identified key indicators that serve as markers for democracy backsliding. With a clear definition of democracy backsliding in place, we then pinpointed countries that have undergone this phenomenon. To enrich our analysis, we incorporated data from reputable sources such as the  World Development Index from World Bank Group, ensuring a comprehensive exploration of economic factors in nations affected by democracy backsliding. The culmination of these efforts resulted in the development of a dynamic dashboard, empowering users to delve into diverse indicators and datasets through a variety of visualizations.

###### Code

In [1]:
# Importing necessary libraries for data manipulation, visualization, and building the dashboard
import pandas as pd
import numpy as np
import plotly.express as px
import dash

In [2]:
# Importing specific modules from Dash for constructing the dashboard components
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output

In [3]:
# Reading the V-Dem dataset and storing it in a DataFrame
df_vdem = pd.read_csv('V-Dem-CY-Full+Others-v13.csv')

/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_8188/2301435376.py:2: DtypeWarning: Columns (364,365,366,394,410,821,831,832,930,1235,1252,1481,3063,3282,3283,3351,3353,3357,3358,3360,3361,3362) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vdem = pd.read_csv('V-Dem-CY-Full+Others-v13.csv')


In [4]:
# Creating pivot tables for specific indicators
# Each pivot table organizes data by country_name and year for the respective indicator
df_regime_1 = df_vdem.pivot(index='country_name', columns='year', values='v2x_regime')
df_dem_1 = df_vdem.pivot(index='country_name', columns='year', values='e_boix_regime')
df_polity_1 = df_vdem.pivot(index='country_name', columns='year', values='e_polity2')

In [5]:
# Dropping columns corresponding to the range from 1789-1991 and years 2019-2022
df_regime_2 = df_regime_1.drop(columns=list(range(1789, 1991)) + [2019, 2020, 2021, 2022])
df_dem_2 = df_dem_1.drop(columns=list(range(1789, 1991)) + [2019, 2020, 2021, 2022])
df_polity_2 = df_polity_1.drop(columns=list(range(1789, 1991)) + [2019, 2020, 2021, 2022])

In [6]:
# Removing rows with missing values (NaN)
df_regime_3 = df_regime_2.dropna()
df_dem_3 = df_dem_2.dropna()
df_polity_3 = df_polity_2.dropna()

In [7]:
# Iterating through rows of the regime pivot table to identify and record regime transition years
regime_transitions_list = []

for index, row in df_regime_3.iterrows():
    country_name = index
    transition_started = False
    
    for year in df_regime_3.columns[:-1]:
        current_value = row[year]
        
        # Checking if the regime value is greater than or equal to 2 and subsequent years have values less than 2
        if (current_value >= 2) and all(row[year + i] < 2 for i in range(1, len(df_regime_3.columns) - df_regime_3.columns.get_loc(year))):
            transition_started = True
            transition_start_year = year

        # Checking if a transition has started and subsequent years have values less than 2
        elif transition_started and all(row[year + i] < 2 for i in range(1, len(df_regime_3.columns) - df_regime_3.columns.get_loc(year))):
            transition_info = {'country': country_name, 'start_year': transition_start_year}
            regime_transitions_list.append(transition_info)
            transition_started = False
            break

In [8]:
# Iterating through rows of the democracy pivot table to identify and record democratization transition years
dem_transitions_list = []

for index, row in df_dem_3.iterrows():
    country_name = index
    transition_started = False
    
    for year in df_dem_3.columns[:-1]:
        current_value = row[year]
        
        # Checking if the democracy value is 1.0 and subsequent years have values of 0.0
        if current_value == 1.0 and all(row[year + i] == 0.0 for i in range(1, len(df_dem_3.columns) - df_dem_3.columns.get_loc(year) - 1)):
            transition_started = True
            transition_start_year = year

        # Checking if a transition has started and subsequent years have values of 0.0
        elif transition_started and all(row[year + i] == 0.0 for i in range(1, len(df_dem_3.columns) - df_dem_3.columns.get_loc(year) - 1)):
            transition_info = {'country': country_name, 'start_year': transition_start_year}
            dem_transitions_list.append(transition_info)
            transition_started = False
            break

In [9]:
# Iterating through rows of the polity pivot table to identify and record transitions in polity scores
polity_transitions_list = []

for index, row in df_polity_3.iterrows():
    country_name = index
    transition_started = False
    
    for year in df_polity_3.columns[:-1]:
        current_value = row[year]
        
        # Checking if the polity score is greater than or equal to 7 and subsequent years have values less than 7
        if (current_value >= 7) and all(row[year + i] < 7 for i in range(1, len(df_polity_3.columns) - df_polity_3.columns.get_loc(year) - 1)):
            transition_started = True
            transition_start_year = year

        # Checking if a transition has started and subsequent years have values less than 7
        elif transition_started and all(row[year + i] < 7 for i in range(1, len(df_polity_3.columns) - df_polity_3.columns.get_loc(year) - 1)):
            transition_info = {'country': country_name, 'start_year': transition_start_year}
            polity_transitions_list.append(transition_info)
            transition_started = False
            break

In [10]:
# Creating sets of unique countries involved in regime, democratization, and polity transitions
regime_set = {entry['country'] for entry in regime_transitions_list}
dem_set = {entry['country'] for entry in dem_transitions_list}
polity_set = {entry['country'] for entry in polity_transitions_list}

In [11]:
# Finding the intersection sets between regime and democratization, regime and polity, and democratization and polity sets
intersection_set_1 = regime_set.intersection(dem_set)
intersection_set_2 = regime_set.intersection(polity_set)
intersection_set_3 = dem_set.intersection(polity_set)

In [12]:
# Creating a union set of countries involved in at least two types of transitions
union_of_sets = (intersection_set_1.union(intersection_set_2)).union(intersection_set_3)

In [13]:
# Filtering regime transitions list to include only entries corresponding to countries in the union set
union_list = [{'country': entry['country'], 'start_year': entry['start_year']} for entry in regime_transitions_list if entry['country'] in union_of_sets]

In [14]:
# Converting the filtered list to a DataFrame for further analysis or visualization
df_union_list = pd.DataFrame(union_list)

In [15]:
# Reading World Development Indicators data
df_wdi_1 = pd.read_excel('P_Data_Extract_From_World_Development_Indicators.xlsx')

In [16]:
# Extracting country names from the union_list of transition information
country_names = [country_dict['country'] for country_dict in union_list]

In [17]:
# Filtering the World Development Indicators DataFrame to include only rows corresponding to countries in the union list
df_wdi_2 = df_wdi_1[df_wdi_1['Country Name'].isin(country_names)]

In [18]:
# Replacing string representations of missing values with actual NaN and dropping rows with missing values
df_wdi_2.replace({'..': np.nan, '...': np.nan}, inplace=True)
df_wdi_2.dropna(inplace=True)

/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_8188/1390409372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wdi_2.replace({'..': np.nan, '...': np.nan}, inplace=True)
/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_8188/1390409372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wdi_2.dropna(inplace=True)


In [19]:
# Identifying common indicators across all countries in the filtered WDI DataFrame
common_indicators = set(df_wdi_2['Series Name'])

In [20]:
# Iterating through countries and finding the intersection of indicators for each country with common indicators
for country in df_wdi_2['Country Name'].unique():
    country_indicators = set(df_wdi_2[df_wdi_2['Country Name'] == country]['Series Name'])
    common_indicators = common_indicators.intersection(country_indicators)

In [21]:
# Filtering the WDI DataFrame to include only rows corresponding to common indicators
df_wdi_3 = df_wdi_2[df_wdi_2['Series Name'].isin(common_indicators)]

In [22]:
# Melting the WDI DataFrame to transform it into a long format
df_wdi_melted = df_wdi_3.melt(id_vars=['Country Name', 'Series Name', 'Series Code'], var_name='Year', value_name='Value')

In [24]:
# Function to create a line plot
def line_plot(data, x_column, y_column, color_column, title):
    fig = px.line(data, x=x_column, y=y_column, color=color_column, title=title)
    fig.update_traces(mode='lines')    
    return fig

# Function to create a scatter plot
def scatter_plot(data, x_column, y_column, color_column, hover_column, animation_column, title, x_label, y_label):
    min_value_x = data[x_column].min()
    max_value_x = data[x_column].max()
    min_value_y = data[y_column].min()
    max_value_y = data[y_column].max()
    fig = px.scatter(data, x=x_column, y=y_column, color=color_column,
                     hover_name=hover_column, animation_frame=animation_column,
                     title=title, animation_group=hover_column,
                     labels={x_column: x_label, y_column: y_label})
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(xaxis=dict(range=[min_value_x, max_value_x]), yaxis=dict(range=[min_value_y, max_value_y]))
    return fig

# Function to create a log-scale scatter plot
def log_scatter_plot(data, x_column, y_column, color_column, hover_column, animation_column, title, x_label, y_label):
    min_value_x = data[x_column].min()
    max_value_x = data[x_column].max()
    min_value_y = data[y_column].min()
    max_value_y = data[y_column].max()
    fig = px.scatter(data, x=x_column, y=y_column, color=color_column,
                     hover_name=hover_column, animation_frame=animation_column, log_x=True, log_y=True,
                     title=title, animation_group=hover_column,
                     labels={x_column: x_label, y_column: y_label})
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(xaxis=dict(range=np.log10([min_value_x, max_value_x])),
                      yaxis=dict(range=np.log10([min_value_y, max_value_y])))
    return fig

In [25]:
# Creating a Dash app instance
app = dash.Dash(__name__)

# Defining the layout of the Dash app
app.layout = html.Div([
    # Header and introduction section
    html.Div(children='Democracy Backsliding and Analysis', style={'textAlign': 'center', 'fontSize': 24, 'fontWeight': 'bold'}),
    dcc.Markdown('''
        ## Understanding Democracy Backsliding
        In examining democracy backsliding across countries spanning the years 1991 to 2018, 
        our project relies on democracy metrics sourced from the Varieties of Democracy (V-Dem) dataset, version 13. 
        A nation is categorized as a democratic backslider under two specific conditions: 
        firstly, it must have attained a notable level of democracy, and subsequently, 
        it should undergo significant erosion of democratic institutions. 
        Our approach to defining a significant level of democracy considers a country to have achieved this status 
        when meeting criteria such as a Regimes of the World (ROW) measure of equal to or greater than 2, 
        a Democracy (BMR) score of 1, and a Polity revised combined score equal to or greater than 7. 
        Similarly, our criteria for democratic decline inclusively encompass countries experiencing a 
        qualitative rating decline or identified as backsliders by at least two major democracy indices. 
        This involves a ROW measure of less than 2, a Democracy (BMR) score of 0, and a Polity revised combined score of less than 7. 
        Based on our approach produces thirteen cases of democratic backsliding since the end of the Cold War.
    ''', style={'textAlign': 'justify'}),
    
    # Table displaying countries and years of democracy backsliding
    html.Div([
        dash_table.DataTable(
            data=df_union_list.to_dict('records'),
            columns=[
                {'name': 'Country', 'id': 'country'},
                {'name': 'Year of Democracy Backsliding', 'id': 'start_year'},
            ],
            style_table={'height': '800px', 'overflowY': 'auto', 'width': '400px', 'margin': '0px'},
            style_cell={'textAlign': 'center'}
        )
    ], style={'display': 'flex', 'justify-content': 'center', 'margin-bottom': '10px'}),
    
    # Choropleth map displaying democracy backsliding over time
    dcc.Graph(
        figure=px.choropleth(
            df_union_list,
            locations='country',
            locationmode='country names',
            hover_name='country',
            color='start_year',
            color_continuous_scale=px.colors.sequential.Plasma,
            title='Democracy Consolidation and Backsliding (Year 1991 to 2019)',
            labels={'start_year': 'Year of Democracy Backsliding', 'country': 'Country'},
        ),
        style={'width': '100%', 'height': '800px', 'margin-top': '-350px'}),
    
    # Introduction to economics analysis section
    dcc.Markdown('''
        ## Economics Analysis
        In order to comprehend the economic context of countries that have undergone democracy backsliding, 
        this section empowers you to delve into specific economic indicators through insightful graphical representations.
        #### Indicator Selection
        To tailor your analysis, select from a diverse array of economic indicators available for visualization. 
        The chosen indicators will be graphically presented, 
        providing a nuanced understanding of their trends and relationships over time.
        #### Graph Types
        Choose from various graph types to suit your analytical needs:
        - **Line Plot:** Trace the evolution of a single economic indicator over time for selected countries. (Note: Please be aware that only the option in the initial dropdown menu will be chosen.)
        - **Scatter Plot:** Investigate the relationship between two chosen economic indicators across countries, with the flexibility of including a time dimension.
        - **Scatter Plot (Log):** Similar to the Scatter Plot, but with a logarithmic scale, offering enhanced visibility for a wide range of values.
        #### Interactivity
        The interactive nature of the graphs allows you to hover over data points, explore specific years, 
        and discern trends with ease. Analyze the economic landscape of countries that have experienced 
        democracy backsliding, fostering a comprehensive understanding of their economic trajectories.
        Feel empowered to engage with the data, uncover insights, and draw meaningful conclusions about 
        the economic dimensions intertwined with democracy backsliding.
    ''', style={'textAlign': 'justify'}),
    
    # Dropdowns for selecting graph type and economic indicators
    dcc.Dropdown(
        id='graph-dropdown',
        options=[
            {'label': 'Line Plot', 'value': 'Line Plot'},
            {'label': 'Scatter Plot', 'value': 'Scatter Plot'},
            {'label': 'Scatter Plot (Log)', 'value': 'Scatter Plot (Log)'}
        ],
        value='Line Plot',
        style={'width': '50%', 'margin-top': '10px'}),

    dcc.Dropdown(
        id='indicator-dropdown-1',
        options=[{'label': indicator, 'value': indicator} for indicator in df_wdi_melted['Series Name'].unique()],
        value=df_wdi_melted['Series Name'].iloc[0],
        style={'width': '50%', 'margin-top': '10px'}
    ),
    
    dcc.Dropdown(
        id='indicator-dropdown-2',
        options=[{'label': indicator, 'value': indicator} for indicator in df_wdi_melted['Series Name'].unique()],
        value=df_wdi_melted['Series Name'].iloc[0],
        style={'width': '50%', 'margin-top': '10px'}
    ),
    
    # Graph displaying selected economic indicators
    dcc.Graph(id='selected-graph', style={'width': '100%', 'height': '600px'})
])

# Callback function to update the selected graph based on user inputs
@app.callback(
    Output('selected-graph', 'figure'),
    [Input('graph-dropdown', 'value'),
     Input('indicator-dropdown-1', 'value'),
     Input('indicator-dropdown-2', 'value')]
)
def update_graph(selected_graph, selected_indicator_1, selected_indicator_2):
    # Filtering data for the selected economic indicators
    df_indicator_1 = df_wdi_melted[df_wdi_melted['Series Name'] == selected_indicator_1]
    df_indicator_2 = df_wdi_melted[df_wdi_melted['Series Name'] == selected_indicator_2]
    
    # Merging data based on common columns (Country Name and Year)
    merged_data = pd.merge(df_indicator_1, df_indicator_2, on=['Country Name', 'Year'])
    
    # Choosing the appropriate plot based on user selection
    if selected_graph == 'Line Plot':
        return line_plot(merged_data, 'Year', 'Value_x', 'Country Name', f'{selected_indicator_1} Over Time')

    elif selected_graph == 'Scatter Plot':
        return scatter_plot(merged_data, 'Value_x', 'Value_y', 'Country Name', 'Country Name', 'Year',
                            f'{selected_indicator_1} vs. {selected_indicator_2}', selected_indicator_1, selected_indicator_2)

    elif selected_graph == 'Scatter Plot (Log)':
        return log_scatter_plot(merged_data, 'Value_x', 'Value_y', 'Country Name', 'Country Name', 'Year',
                                f'{selected_indicator_1} vs. {selected_indicator_2} (Log Scale)', selected_indicator_1, selected_indicator_2)

# Running the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=True, open_browser=True)

###### Conclusion

In conclusion, our Python project provides a comprehensive economic analysis of countries that have experienced democracy backsliding, spanning the years 1991 to 2018. Leveraging data from the V-Dem dataset, our approach defines backsliding based on multiple criteria, including Regimes of the World measures, Democracy (BMR) scores, and Polity revised combined scores. The identification of countries experiencing significant erosion of democratic institutions, as well as a detailed exploration of economic indicators through World Development Indicators, offers valuable insights into the complex interplay between political and economic factors.

The visualization tools incorporated into our Dash app, such as choropleth maps, line plots, and scatter plots, enhance the user's ability to understand and interpret the data. The project not only highlights the countries that have undergone democracy backsliding but also allows users to explore the economic context of these transitions.

This Python project showcases the power of data analysis and visualization in understanding complex socio-political phenomena. The combination of rigorous data processing, insightful criteria definition, and dynamic visualization tools provides a robust platform for researchers, policymakers, and the public to delve into the nuanced dynamics of democracy and its intersections with economic indicators.
